In [118]:
import pandas as pd
import numpy as np
import json

In [124]:
f = open('5492147125.json')

In [125]:
raw_data = json.load(f)

In [135]:
for key in raw_data["frames"][1:]["Events"]:
    print(key)

TypeError: list indices must be integers or slices, not str

In [127]:
map_info = raw_data["frames"][0]

In [128]:
map_info.keys()

dict_keys(['DataVersion', 'Entities', 'Events'])

In [129]:
first_frame = raw_data["frames"][2500]["Entities"]

In [130]:
df =  pd.DataFrame(first_frame).T

In [131]:
df

,UID,ENTITY_TYPE,ID,GAME_TIME,CLOCK_TIME,TIME_IN_PHASE,MILLISECONDS_SINCE_GAME_START,FRAMEINDEX,KEYFRAME,RadiantLastHits,...,YPOS,MANA,MANA_PERCENT,HEALTH_REGEN,MANA_REGEN,TOTAL_AGILITY,DAMAGE_MIN,DAMAGE_MAX,DAMAGE_BONUS,COOLDOWN_LENGTH
0000000000,0,Dota2MapEntity,MAP,2509.9282,1540.2214,1540.2214,2523000.0,2500,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435459,268435459,Dota2MapRegion,JUNGLE_RADIANTBOTTOM,NaN,NaN,NaN,NaN,NaN,NaN,78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435463,268435463,Dota2MapRegion,LANE_MIDDLE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435466,268435466,PlayerEntity,PLAYER0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435467,268435467,PlayerEntity,PLAYER1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435468,268435468,PlayerEntity,PLAYER2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435469,268435469,PlayerEntity,PLAYER3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435470,268435470,PlayerEntity,PLAYER4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435471,268435471,PlayerEntity,PLAYER5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435472,268435472,PlayerEntity,PLAYER6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
df[df['ENTITY_TYPE']=="HeroEntity"].dropna(axis=1)

,UID,ENTITY_TYPE,ID,XPOS,YPOS
0268435723,268435723,HeroEntity,HERO0,-397.5,-4021.9375
0268435724,268435724,HeroEntity,HERO1,-1689.0,5784.5625
0268435725,268435725,HeroEntity,HERO2,-74.6875,-2455.125
0268435726,268435726,HeroEntity,HERO3,6887.0,-5147.0
0268435727,268435727,HeroEntity,HERO4,-2393.28125,1158.28125
0268435728,268435728,HeroEntity,HERO5,1307.25,5468.6875
0268435729,268435729,HeroEntity,HERO6,5143.4375,5204.3125
0268435730,268435730,HeroEntity,HERO7,1585.375,2321.0
0268435732,268435732,HeroEntity,HERO9,1531.125,1232.53125


In [95]:
def get_player_from_frame(frame_number):
    entity_data = raw_data["frames"][frame_number]["Entities"]
    df =  pd.DataFrame(entity_data).T
    player_df = df[df['ENTITY_TYPE']=="PlayerEntity"].dropna(axis=1)
    hero_df = df[df['ENTITY_TYPE']=="HeroEntity"].dropna(axis=1)
    hero_df['ID'] = hero_df['ID'].apply(lambda x: x.replace('HERO', 'PLAYER'))
    result = pd.concat([hero_df.set_index('ID'),player_df.set_index('ID')], axis=1, join='inner')
    result = result.drop("ENTITY_TYPE",axis=1)
    result.set_index(result.index)
    result["UIDF"] = [f"{x}{frame_number}" for x in range(0,len(result))]
    result.set_index("UIDF",inplace=True)
    return result

In [101]:
get_player_from_frame(2500)

,UID,XPOS,YPOS,UID,NET_WORTH,GOLD,GOLD_EARNED,GOLD_RELIABLE,GOLD_FROM_INCOME,XCURSOR,YCURSOR
UIDF,,,,,,,,,,,
02500,268435723,-397.5,-4021.9375,268435466,13786.0,1891.0,13742.0,480.0,2430.0,324.0,180.0
12500,268435724,-1689.0,5784.5625,268435467,11726.0,276.0,12178.0,115.0,2430.0,227.0,103.0
22500,268435725,-74.6875,-2455.125,268435468,8817.0,1377.0,9758.0,1021.0,2430.0,282.0,212.0
32500,268435726,6887.0,-5147.0,268435469,3586.0,1081.0,5431.0,904.0,2430.0,286.0,153.0
42500,268435727,-2393.28125,1158.28125,268435470,8469.0,1004.0,9228.0,509.0,2430.0,162.0,90.0
52500,268435728,1307.25,5468.6875,268435471,5829.0,1589.0,7894.0,617.0,2430.0,159.0,203.0
62500,268435729,5143.4375,5204.3125,268435472,10184.0,924.0,10534.0,613.0,2430.0,34.0,339.0
72500,268435730,1585.375,2321.0,268435473,7707.0,757.0,8133.0,516.0,2430.0,328.0,196.0
82500,268435732,1531.125,1232.53125,268435475,15516.0,891.0,16345.0,149.0,2430.0,329.0,124.0


In [102]:
frames = list(range(2500,2504))

In [104]:
frame_dfs =  [get_player_from_frame(x) for x in frames]

In [107]:
indexes = []

for frame in frame_dfs:
    indexes+=list(frame.index)
print(len(indexes))
print(len(set(indexes)))

36
36


In [113]:
frame_dfs[0]

,UID,XPOS,YPOS,UID,NET_WORTH,GOLD,GOLD_EARNED,GOLD_RELIABLE,GOLD_FROM_INCOME,XCURSOR,YCURSOR
UIDF,,,,,,,,,,,
02500,268435723,-397.5,-4021.9375,268435466,13786.0,1891.0,13742.0,480.0,2430.0,324.0,180.0
12500,268435724,-1689.0,5784.5625,268435467,11726.0,276.0,12178.0,115.0,2430.0,227.0,103.0
22500,268435725,-74.6875,-2455.125,268435468,8817.0,1377.0,9758.0,1021.0,2430.0,282.0,212.0
32500,268435726,6887.0,-5147.0,268435469,3586.0,1081.0,5431.0,904.0,2430.0,286.0,153.0
42500,268435727,-2393.28125,1158.28125,268435470,8469.0,1004.0,9228.0,509.0,2430.0,162.0,90.0
52500,268435728,1307.25,5468.6875,268435471,5829.0,1589.0,7894.0,617.0,2430.0,159.0,203.0
62500,268435729,5143.4375,5204.3125,268435472,10184.0,924.0,10534.0,613.0,2430.0,34.0,339.0
72500,268435730,1585.375,2321.0,268435473,7707.0,757.0,8133.0,516.0,2430.0,328.0,196.0
82500,268435732,1531.125,1232.53125,268435475,15516.0,891.0,16345.0,149.0,2430.0,329.0,124.0


In [116]:
frame_dfs[0].append(frame_dfs[1:])

InvalidIndexError: Reindexing only valid with uniquely valued Index objects

In [23]:
df1 = df[df['ENTITY_TYPE']=="PlayerEntity"].dropna(axis=1)
df1

,UID,ENTITY_TYPE,ID,NET_WORTH,GOLD,GOLD_EARNED,GOLD_RELIABLE,GOLD_FROM_INCOME,XCURSOR
0268435466,268435466,PlayerEntity,PLAYER0,18549.0,1564.0,19494.0,176.0,3220.0,308.0
0268435467,268435467,PlayerEntity,PLAYER1,12685.0,1235.0,15436.0,136.0,3220.0,364.0
0268435468,268435468,PlayerEntity,PLAYER2,12055.0,1840.0,13715.0,700.0,3220.0,309.0
0268435469,268435469,PlayerEntity,PLAYER3,4420.0,2065.0,7470.0,1804.0,3220.0,93.0
0268435470,268435470,PlayerEntity,PLAYER4,10040.0,1075.0,12028.0,678.0,3220.0,498.0
0268435471,268435471,PlayerEntity,PLAYER5,8566.0,2076.0,11857.0,635.0,3220.0,122.0
0268435472,268435472,PlayerEntity,PLAYER6,15396.0,1706.0,16543.0,815.0,3220.0,239.0
0268435473,268435473,PlayerEntity,PLAYER7,10556.0,206.0,12827.0,206.0,3220.0,38.0
0268435474,268435474,PlayerEntity,PLAYER8,5785.0,370.0,8039.0,370.0,3220.0,328.0
0268435475,268435475,PlayerEntity,PLAYER9,23867.0,777.0,25131.0,664.0,3220.0,230.0


In [39]:
df2 = df[df['ENTITY_TYPE']=="HeroEntity"]
df2

,UID,ENTITY_TYPE,ID,GAME_TIME,CLOCK_TIME,TIME_IN_PHASE,MILLISECONDS_SINCE_GAME_START,FRAMEINDEX,KEYFRAME,RadiantLastHits,...,YPOS,MANA,MANA_PERCENT,HEALTH_REGEN,MANA_REGEN,TOTAL_AGILITY,DAMAGE_MIN,DAMAGE_MAX,DAMAGE_BONUS,COOLDOWN_LENGTH
0268435723,268435723,HeroEntity,HERO0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-4021.9375,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435724,268435724,HeroEntity,HERO1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5784.5625,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435725,268435725,HeroEntity,HERO2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-2455.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435726,268435726,HeroEntity,HERO3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-5147.0,365.0,73.88664,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435727,268435727,HeroEntity,HERO4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1158.28125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435728,268435728,HeroEntity,HERO5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5468.6875,620.0,74.698795,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435729,268435729,HeroEntity,HERO6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5204.3125,NaN,NaN,172.31967,NaN,NaN,NaN,NaN,NaN,NaN
0268435730,268435730,HeroEntity,HERO7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2321.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0268435732,268435732,HeroEntity,HERO9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1232.53125,622.0,82.058047,NaN,6.366356,155.800018,181,187,85,NaN


In [19]:
df


NameError: name 'df1' is not defined

In [74]:
for frame_number,frame_val in first_frame.items():
    print(frame_val["ID"])

MAP
PLAYER3
PLAYER4
PLAYER6
PLAYER9


In [32]:
df = pd.DataFrame(raw_data["frames"][50]["Entities"])
df.T


,UID,ENTITY_TYPE,ID,GAME_TIME,CLOCK_TIME,TIME_IN_PHASE,MILLISECONDS_SINCE_GAME_START,FRAMEINDEX,KEYFRAME,XCURSOR,YCURSOR
0000000000,0,Dota2MapEntity,MAP,58.766113,58.766113,58.766113,51000.0,50,False,NaN,NaN
0268435466,268435466,PlayerEntity,PLAYER0,NaN,NaN,NaN,NaN,NaN,NaN,289.0,148.0
0268435467,268435467,PlayerEntity,PLAYER1,NaN,NaN,NaN,NaN,NaN,NaN,303.0,145.0
0268435468,268435468,PlayerEntity,PLAYER2,NaN,NaN,NaN,NaN,NaN,NaN,306.0,NaN
0268435469,268435469,PlayerEntity,PLAYER3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
0268435470,268435470,PlayerEntity,PLAYER4,NaN,NaN,NaN,NaN,NaN,NaN,312.0,39.0
0268435472,268435472,PlayerEntity,PLAYER6,NaN,NaN,NaN,NaN,NaN,NaN,273.0,189.0
0268435473,268435473,PlayerEntity,PLAYER7,NaN,NaN,NaN,NaN,NaN,NaN,331.0,138.0
0268435475,268435475,PlayerEntity,PLAYER9,NaN,NaN,NaN,NaN,NaN,NaN,259.0,135.0


,DataVersion,Entities,Events
0000000000,1.0,"{'UID': 0, 'ENTITY_TYPE': 'Dota2MapEntity', 'I...",NaN
0268435456,1.0,"{'UID': 268435456, 'ENTITY_TYPE': 'Dota2MapReg...",NaN
0268435457,1.0,"{'UID': 268435457, 'ENTITY_TYPE': 'Dota2MapReg...",NaN
0268435458,1.0,"{'UID': 268435458, 'ENTITY_TYPE': 'Dota2MapReg...",NaN
0268435459,1.0,"{'UID': 268435459, 'ENTITY_TYPE': 'Dota2MapReg...",NaN
0268435460,1.0,"{'UID': 268435460, 'ENTITY_TYPE': 'Dota2MapReg...",NaN
0268435461,1.0,"{'UID': 268435461, 'ENTITY_TYPE': 'Dota2MapReg...",NaN
0268435462,1.0,"{'UID': 268435462, 'ENTITY_TYPE': 'Dota2MapReg...",NaN
0268435463,1.0,"{'UID': 268435463, 'ENTITY_TYPE': 'Dota2MapReg...",NaN
0268435464,1.0,"{'UID': 268435464, 'ENTITY_TYPE': 'Dota2MapReg...",NaN
